In [45]:
%matplotlib widget
from requests import get
import pandas as df
import numpy as np
def get_data(url):
    response = get(endpoint, timeout=10)
    
    if response.status_code >= 400:
        raise RuntimeError(f'Request failed: { response.text }')
        
    return response.json()
    

endpoint = (
    'https://api.coronavirus.data.gov.uk/v1/data?'
    'filters=areaType=nation&'
    'structure={"date":"date","areaName":"areaName","areaCode":"areaCode","newCasesByPublishDate":"newCasesByPublishDate","cumCasesByPublishDate":"cumCasesByPublishDate","newDeathsByDeathDate":"newDeathsByDeathDate","cumDeathsByDeathDate":"cumDeathsByDeathDate"}'
)

data = get_data(endpoint)
print(data['data'])

[{'date': '2020-11-23', 'areaName': 'England', 'areaCode': 'E92000001', 'newCasesByPublishDate': 13329, 'cumCasesByPublishDate': 1314888, 'newDeathsByDeathDate': None, 'cumDeathsByDeathDate': None}, {'date': '2020-11-22', 'areaName': 'England', 'areaCode': 'E92000001', 'newCasesByPublishDate': 16668, 'cumCasesByPublishDate': 1301559, 'newDeathsByDeathDate': 66, 'cumDeathsByDeathDate': 58428}, {'date': '2020-11-21', 'areaName': 'England', 'areaCode': 'E92000001', 'newCasesByPublishDate': 17615, 'cumCasesByPublishDate': 1284891, 'newDeathsByDeathDate': 132, 'cumDeathsByDeathDate': 58362}, {'date': '2020-11-20', 'areaName': 'England', 'areaCode': 'E92000001', 'newCasesByPublishDate': 17845, 'cumCasesByPublishDate': 1267276, 'newDeathsByDeathDate': 262, 'cumDeathsByDeathDate': 58230}, {'date': '2020-11-19', 'areaName': 'England', 'areaCode': 'E92000001', 'newCasesByPublishDate': 20291, 'cumCasesByPublishDate': 1249431, 'newDeathsByDeathDate': 349, 'cumDeathsByDeathDate': 57968}, {'date': '

In [54]:
# df.DataFrame.from_records(data)
# train = df.DataFrame.from_dict(data['data'], orient='index')
from pandas.io.json import json_normalize
dataframe=json_normalize(data['data'])
dataframe=dataframe.reset_index(drop= True)

c:\users\haddi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


In [55]:
import matplotlib.pyplot as plt
dataframe['date']=dataframe['date'].astype('datetime64')
dataframe['date'].count()

1304

In [56]:
# plt.bar(dataframe['date'],dataframe['newCasesByPublishDate'])
dataframe=dataframe.dropna(axis=0).reset_index(drop= True)

In [57]:
# plt.bar(dataframe['date'],dataframe['cumDeathsByDeathDate'])


In [58]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [59]:
from datetime import datetime
from IPython.display import display
start_date = dataframe['date'].min()
end_date = dataframe['date'].max()

dates = df.date_range(start_date, end_date, freq='D')

options = [(date.strftime(' %d %b %Y '), date) for date in dates]
index = (0, len(options)-1)

# selection_range_slider = widgets.SelectionRangeSlider(
#     options=options,
#     index=index,
#     description='Dates',
#     orientation='horizontal',
#     layout={'width': '500px'}
# )

# def print_date_range(date_range):
#     condition=(dataframe['date']>date_range[0]) & (dataframe['date']<date_range[1])
#     chunk_df=dataframe[condition]
#     plt.bar(chunk_df['date'],chunk_df['cumDeathsByDeathDate'])
#     plt.show()
# #     print(dataframe[dataframe['date']>date_range[0]])

# widgets.interact(
#     print_date_range,
#     date_range=selection_range_slider,
#     continuous_update=False
# );

In [60]:
# print(dataframe['date'], dataframe['cumDeathsByDeathDate'])

In [68]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )
 
class Sines(widgets.HBox):
     
    def __init__(self,data):
        super().__init__()
        output = widgets.Output()
        self.dataframe=data
        self.updated_data=data
        self.x = 'date'
        self.y='cumDeathsByDeathDate'
        initial_color = '#FF00DD'
 
        with output:
            self.fig, self.ax = plt.subplots(constrained_layout=True, figsize=(5, 3.5))
        self.line, = self.ax.plot(self.dataframe['date'], self.dataframe['cumDeathsByDeathDate'], initial_color)
         
        self.fig.canvas.toolbar_position = 'bottom'
        self.ax.grid(True)
 
        # define widgets
        int_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Dates',
    orientation='horizontal',
    layout={'width': '500px'}
)
        color_picker = widgets.ColorPicker(
            value=initial_color, 
            description='pick a color'
        )
        column_x = widgets.Dropdown(
    options=self.dataframe.columns,
    value=self.dataframe.columns[0],
    description='Y axis:',
)
        text_xlabel = widgets.Text(
            value='', 
            description='xlabel', 
            continuous_update=False
        )
        text_ylabel = widgets.Text(
            value='', 
            description='ylabel', 
            continuous_update=False
        )
 
        controls = widgets.VBox([
            int_slider, 
            color_picker,
            column_x,
            text_xlabel, 
            text_ylabel
        ])
        controls.layout = make_box_layout()
         
        out_box = widgets.Box([output])
        output.layout = make_box_layout()
 
        # observe stuff

        column_x.observe(self.on_change)
        int_slider.observe(self.update, 'value')
        color_picker.observe(self.line_color, 'value')
        text_xlabel.observe(self.update_xlabel, 'value')
        text_ylabel.observe(self.update_ylabel, 'value')
         
        text_xlabel.value = self.x
        text_ylabel.value = self.y
         
 
        # add to children
        self.children = [controls, output]
     
    def update(self, date_range):
#         print(date_range.new)
        """Draw line in plot"""
        x=self.dataframe['date']>date_range.new[0]
        y=self.dataframe['date']<date_range.new[1]
        condition=x & y
        chunk_df=self.dataframe[condition]
        self.updated_data=chunk_df.copy()
#         print((self.dataframe['date']>date_range.new[0]))
        self.line.set_ydata(self.updated_data[self.y])
        
        self.line.set_xdata(self.updated_data[self.x])
        self.fig.canvas.draw()
 
    def line_color(self, change):
        self.line.set_color(change.new)
 
    def update_xlabel(self, change):
        self.ax.set_xlabel(change.new)
 
    def update_ylabel(self, change):
        self.ax.set_ylabel(change.new)
    def on_change(self,change):
        if change['type'] == 'change' and change['name'] == 'value':
            self.line.set_ydata(self.updated_data[change['new']])
            self.ax.set_ylabel(change['new'])
            self.y=change['new']

 
         
Sines(dataframe)

Sines(children=(VBox(children=(SelectionRangeSlider(description='Dates', index=(0, 265), layout=Layout(width='…